# Gather relevant data 

## Webscrape demographic data 

Import Relevant packages

In [1]:
import pandas as pd
import numpy as np
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

Checks to confirm page exists

In [2]:
url = 'https://www.zip-codes.com{}'
columbus_branch ='/city/oh-columbus.asp'
# reinforces integrity of the data collection
accessed = False
while accessed == False:
    try:
        response = requests.get(url.format(columbus_branch))
        print(response)
        accessed = True
    except:
        accesssed = False

<Response [200]>


Gathers relevant data on the page

In [3]:
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    tbl = soup.table.tr.find_all('td')[1].find_all('table')[1]
    print(tbl)

<table border="0" cellpadding="0" cellspacing="0" class="statTable" id="tblZIP" width="99%">
<tr><td class="label"><strong>ZIP Code</strong></td><td><strong>Type</strong></td><td><strong>County</strong></td><td><strong>Population</strong></td><td><strong>Area Code(s)</strong></td></tr>
<tr><td><a href="/zip-code/43081/zip-code-43081.asp" style="text-decoration:underline;" title="ZIP Code 43081, WESTERVILLE, OH">ZIP Code 43081</a></td><td>Standard</td><td><a href="/county/oh-franklin.asp">Franklin</a></td><td>55,991</td><td><a href="/area-code/area-code-614.asp" title="Area Code 614">614</a></td></tr><tr><td><a href="/zip-code/43085/zip-code-43085.asp" style="text-decoration:underline;" title="ZIP Code 43085, COLUMBUS, OH">ZIP Code 43085</a></td><td>Standard</td><td><a href="/county/oh-franklin.asp">Franklin</a></td><td>23,258</td><td><a href="/area-code/area-code-614.asp" title="Area Code 614">614</a> / <a href="/area-code/area-code-380.asp" title="Area Code 380">380</a> / <a href="/ar

function to help web scrape zip code information

In [7]:
# @TODO: Fix zip info extraction method.
# utilize when using the table the [0-4] table 
# BeautifulSoup(response.text, 'html.parser').table.tr.find_all('td')[1].find_all('table')
def zip_info(response):
    page = BeautifulSoup(response.text, 'html.parser').tr#.find_all('td')[1]
    #print(page)
    dict = {}
    for table in page.find_all('table'):
        for row in table.find_all('tr'):
            dict[row.find_all('td')[0].string] = row.find_all('td')[1].string        
    return dict

Parses through and gathers data

In [6]:
i=0
zip_info_df = pd.DataFrame()
zip_url = '/zip-code/{}/zip-code-{}.asp'
for row in tbl.find_all('tr'):
    #checks for header
    if row.find_all('td')[0].string != 'ZIP Code':
        if row.find_all('td')[1].string == 'Standard':
            zip_code = row.find_all('td')[0].string.strip(" ZIPCode")
            print(zip_code)
            #coded just to make sure that we can access the website
            accessed = False
            while accessed == False:
                try:
                    zip_response = requests.get(url.format(zip_url.format(zip_code,zip_code)))
                    accessed = True
                    i+=1
                    zip_dict = zip_info(zip_response)
                    zip_info_df = zip_info_df.append(zip_dict, ignore_index=True)
                except:
                    accessed = False
print(i)
print(zip_info_df)
# reads in all zip code responses

43081
43085
43201
43202
43203
43204
43205
43206
43207
43209
43210
43211
43212
43213
43214
43215
43217
43219
43220
43221
43222
43223
43224
43227
43228
43229
43230
43231
43232
43235
43240
31
Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]


In [10]:
zip_info_df.dtypes

Series([], dtype: object)

## Location data for Hospitals and Pharmacies

Get relevant packages

In [12]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# !conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


Credentials for foursquare_API

In [17]:
# @hidden_cell
CLIENT_ID = '3PRTA5P0AY2A12Z51Z1UHCZH124K5AYLU4PGQHQ0BMCV5FWD' # your Foursquare ID
CLIENT_SECRET = 'JOQYBGLRRR0QNOYE43VOVALU1DFGK4TGSR23SS3J53X2G4JW' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 60
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3PRTA5P0AY2A12Z51Z1UHCZH124K5AYLU4PGQHQ0BMCV5FWD
CLIENT_SECRET:JOQYBGLRRR0QNOYE43VOVALU1DFGK4TGSR23SS3J53X2G4JW


Information for using foursquare API 

In [19]:
zip_info_df.columns

Index([], dtype='object')

In [18]:
CAT_IDs = {'Hospital':'4bf58dd8d48988d196941735', 'Pharmacy': '4bf58dd8d48988d10f951735'}
geo_coords = zip_info_df[['Latitude:','Longitude:']]
radius = 10000 #about 6.2 miles

KeyError: "None of [Index(['Latitude:', 'Longitude:'], dtype='object')] are in the [columns]"

Create parameters that are not hidden

In [20]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&categoryId={}&v={}&radius={}&limit={}'
url

'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&categoryId={}&v={}&radius={}&limit={}'

Grabs data for all hospitals and pharmacies

In [21]:
i=0
# Hospital and Pharmacy Data Frame
hp_df = pd.DataFrame()
while i<len(geo_coords): 
    lat = geo_coords.iloc[i]['Latitude:']
    long = geo_coords.iloc[i]['Longitude:']
    #hospitals
    temp_url = url.format(CLIENT_ID, CLIENT_SECRET, lat, long, CAT_IDs['Hospital'], VERSION, radius, LIMIT)
    results = requests.get(temp_url).json()
    hps = results['response']['venues']
    hp_df=hp_df.append(json_normalize(hps), ignore_index=True)
    #pharmacies
    temp_url = url.format(CLIENT_ID, CLIENT_SECRET, lat, long, CAT_IDs['Pharmacy'], VERSION, radius, LIMIT)
    results = requests.get(temp_url).json()
    hps = results['response']['venues']
    hp_df=hp_df.append(json_normalize(hps), ignore_index=True)
    i+=1
hp_df

NameError: name 'geo_coords' is not defined

Filters to get unique Pharmacy and Hospital Locations

In [14]:
hp_df=hp_df.drop_duplicates(['id'])
hp_df

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '5032891291d4c4b30a586d68', 'name': 'A...",False,4ed664e22fb6cc941132ed13,215 Huber Village Blvd,US,Westerville,United States,NaN,1650,"[215 Huber Village Blvd, Westerville, OH 43081...","[{'label': 'display', 'lat': 40.102667, 'lng':...",40.102667,-82.916989,NaN,43081,OH,The Village At Westerville,v-1598119890,NaN
1,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4c3763da1e06d13ae6d7753e,454 Lazelle Rd,US,Columbus,United States,Sancus,8913,"[454 Lazelle Rd (Sancus), Columbus, OH 43240, ...","[{'label': 'display', 'lat': 40.136307, 'lng':...",40.136307,-82.995064,NaN,43240,OH,Lifetime Pet Wellness,v-1598119890,68773130
2,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,501d17b8e4b09fbdec1c12c9,NaN,US,Columbus,United States,NaN,11751,"[Columbus, OH 43223, United States]","[{'label': 'display', 'lat': 40.11014760015202...",40.110148,-83.036035,NaN,43223,OH,H12B HEM/ONC At NCH,v-1598119890,NaN
3,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,5dfd0cb4028e6200087814c4,4049 The Strand E,US,Columbus,United States,NaN,6156,"[4049 The Strand E, Columbus, OH 43219, United...","[{'label': 'display', 'lat': 40.0523189, 'lng'...",40.052319,-82.915226,NaN,43219,OH,Warby Parker,v-1598119890,566857031
4,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4b391ed3f964a520985625e3,500 S Cleveland Ave,US,Westerville,United States,NaN,4270,"[500 S Cleveland Ave, Westerville, OH 43081, U...","[{'label': 'display', 'lat': 40.11448386021365...",40.114484,-82.947050,NaN,43081,OH,Mount Carmel St. Ann's Hospital,v-1598119890,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",False,5d9e9ac2d517730008146b28,4004 Presidential Pkwy,US,Powell,United States,NaN,9474,"[4004 Presidential Pkwy, Powell, OH 43065, Uni...","[{'label': 'display', 'lat': 40.14919, 'lng': ...",40.149190,-83.091280,NaN,43065,OH,Powell Pharmacy,v-1598119914,NaN
2819,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",False,5824c435cb369f081cee397a,7730 Sawmill Road,US,Dublin,United States,NaN,9512,"[7730 Sawmill Road, Dublin, OH 43016, United S...","[{'label': 'display', 'lat': 40.12437938144518...",40.124379,-83.088128,NaN,43016,OH,Walmart Pharmacy,v-1598119914,NaN
2826,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",False,4c6ee2b69c6d6dcb75d7cd7a,1202 County Line Rd,US,Westerville,United States,NaN,8101,"[1202 County Line Rd, Westerville, OH 43081, U...","[{'label': 'display', 'lat': 40.13218479219031...",40.132185,-82.886545,NaN,43081,OH,CVS pharmacy,v-1598119914,NaN
2840,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",False,4e3af8321838961aff06a1d0,NaN,US,Lewis Center,United States,NaN,3754,"[Lewis Center, OH, United States]","[{'label': 'display', 'lat': 40.16502, 'lng': ...",40.165020,-83.016482,NaN,NaN,OH,Dr. Scholl's Custon Fit Orthotic Center,v-1598119914,NaN


## COVID-19 Numbers

In [21]:
import requests
from bs4 import BeautifulSoup
import json
import re
#TODO figure out how to webscrape COVID-19 data from graph
r = requests.get(
    f"https://public.tableau.com/views/COVID-19OutbreakSummary_15918845768300/COVID19Summaryp3",
     params= {
        ":embed":"y",
        ":showAppBanner":"false",
        ":showShareOptions":"true",
        ":display_count":"no",
    }
)
print(r.url)
soup = BeautifulSoup(r.text, "html.parser")

tableauData = json.loads(soup.find("textarea",{"id": "tsConfigContainer"}).text)

dataUrl = f'https://public.tableau.com{tableauData["vizql_root"]}/bootstrapSession/sessions/{tableauData["sessionid"]}'
r = requests.post(dataUrl, data= {
    "sheet_id": tableauData["sheetId"],
})
dataReg = re.search('\d+;({.*})\d+;({.*})', r.text, re.MULTILINE)
info = json.loads(dataReg.group(1))
data = json.loads(dataReg.group(2))
#print(info)
print(data["secondaryInfo"]["presModelMap"]["dataDictionary"]["presModelHolder"]["genDataDictionaryPresModel"]["dataSegments"]["0"]["dataColumns"])
#<_sre.SRE_Match object; span=(0, 109884), match='72955;{"sheetName":"DASH: Patients v discharges",>

https://public.tableau.com/views/COVID-19OutbreakSummary_15918845768300/COVID19Summaryp3?:embed=y&:showAppBanner=false&:showShareOptions=true&:display_count=no&:showVizHome=no
https://public.tableau.com/vizql/w/COVID-19OutbreakSummary_15918845768300/v/COVID19Summaryp3/bootstrapSession/sessions/9FA0ECDB23A049EFBA7461A60D16D595-0:0
{'secondaryInfo': {}}


## COVID-19 Testing Locations

Install package

In [15]:
import PyPDF2
# source of pdf scraping https://www.youtube.com/watch?v=UmPe07a3bWs

Scrapes pdf itself

In [16]:
import io

import requests
from PyPDF2 import PdfFileReader

url = 'https://www.columbus.gov/uploadedFiles/Columbus/Departments/Public_Health/All_Programs/Emergency_Preparedness/Coronavirus/LocalTesting_FQHCs_7.16.2020.pdf'

r = requests.get(url)
f = io.BytesIO(r.content)

reader = PdfFileReader(f)
contents = reader.getPage(0).extractText().split('\n')
print(contents)

['COVID-19 Community Testing', 'No one will be turned away for inability to pay. Questions? Call Columbus Public Health at 614-645-1519.', '˜˚˛˝˙ˆˇ˘ˆ', '˜', '˜', 'Information compiled by ', 'Columbus Public Health; ', '7/16/2020', 'Columbus Public Health', ' (testing provided ', 'by Mount Carmel, OhioHealth and The Ohio ', 'State University Wexner Medical Center)', '614-645-1519', 'www.columbus.gov/coronavirus', '1. Columbus Public Health', '240 Parsons Ave., Columbus', 'Mon.-Fri. Œ 9 a.m.-4 p.m.', 'Heart of Ohio Family Health ', '614-235-5555', 'https://heartofohiofamilyhealth.org', '2. Capital Park Family Health Center ', '2365 Innis Rd., Columbus', 'Mon., Tue., Wed., Fri. Œ 8 a.m.-5 p.m.', 'Thu., Œ 8 a.m.-7:30 p.m.', '3. Whitehall Family Health Center ', '882 S. Hamilton Rd., Whitehall', 'Mon., Wed., Thu., Fri. Œ 8 a.m.-5 p.m.', 'Tue. Œ 8 a.m.-7:30 p.m.', 'Lower Lights Christian Health Center ', '614-274-1455', 'https://llchc.org/covid19', '4. Lower Lights at Mount Carmel West ', '7

Filters to grab address

In [36]:
import re

address_extract ='\d+[ ](?:[A-Za-z0-9.-]+[ ]?)+(?:Avenue|Lane|Road|Boulevard|Drive|Street|Ave|Dr|Rd|Blvd|Ln|St)\.?'
p = re.compile(address_extract)
testingLocations = [ content for content in contents if p.match(content)]
p2 = re.compile('\d+\.')
covid_test_providers = [ content for content in contents if p2.match(content)]
testingLocations = testingLocations+[re.sub("\d+\.","",test) for test in covid_test_providers if re.search(address_extract,test)]
print(testingLocations)

['240 Parsons Ave., Columbus', '2365 Innis Rd., Columbus', '882 S. Hamilton Rd., Whitehall', '777 W. State St., Columbus', '2300 W. Broad St., Columbus', '1905 Parsons Ave., Columbus', '3433 Agler Rd., Columbus', '1500 E. 17th Ave., Columbus', '750 E. Long St., Columbus', ' 86 N. Front St., Columbus']


# Awesome Selenium Webscraping Action

In [1]:
from selenium import webdriver 
import time
# import Action chains  
from selenium.webdriver.common.action_chains import ActionChains 
driver = webdriver.Firefox()

# Navigate to url
driver.get("https://public.tableau.com/views/COVID-19OutbreakSummary_15918845768300/COVID19Summaryp3?%3Adisplay_count=y&%3Aorigin=viz_share_link&%3AshowVizHome=no")

# Store 'google search' button web element
time.sleep(20)
mapfre = driver.find_element_by_xpath("//*[@id=\"tabZoneId106\"]/div/div/div/div")#//*[@id=\"view8239885986803340325_964909730081303188\"]/div[1]/div[2]")
#Set x and y offset positions of element
xOffset = 0 
yOffset = 0
ActionChains(driver).move_to_element(mapfre).perform()
ActionChains(driver).move_by_offset(xOffset,yOffset).perform()
time.sleep(12)
with open('temp_covid_zip.txt', 'w', encoding='utf-8') as f:
    f.write(str(driver.page_source))

In [2]:
#this script works for extracting zip and cases range
temp = ''
with open('temp_covid_zip.txt', 'r') as fin:
    temp = fin.read()
from bs4 import BeautifulSoup
soup = BeautifulSoup(temp, 'html.parser')
table = soup.find_all('div')[len(soup.find_all('div'))-3].table #grabs the table which stores the data
print(table.find_all('tr')[0].find_all('td')[2].string)
print(table.find_all('tr')[1].find_all('td')[2].string)

43211
245 - 472


In [ ]:
#Use for Y: -39 to start off with to go from 43211 to 43224